In [4]:
import spacy
import huspacy

In [5]:
nlp = huspacy.load()

In [6]:
nlp = spacy.load("hu_core_news_lg")

In [7]:
import hu_core_news_lg
nlp = hu_core_news_lg.load()

In [8]:
doc = nlp("csak azt szerettem volna megkérdezni hogy lenne e kedved eljönni velem ma, hozzám.")

In [9]:
for token in doc:
    print(f"{token.text:15} | {token.lemma_:10} | {token.pos_:6} | {token.dep_}")

csak            | csak       | ADV    | advmod:mode
azt             | az         | PRON   | obj
szerettem       | szeret     | VERB   | ROOT
volna           | volna      | AUX    | aux
megkérdezni     | megkérdez  | VERB   | xcomp
hogy            | hogy       | SCONJ  | mark
lenne           | lesz       | VERB   | ccomp:obj
e               | e          | DET    | det
kedved          | kedv       | NOUN   | nsubj
eljönni         | eljön      | VERB   | xcomp
velem           | vele       | PRON   | obl
ma              | ma         | ADV    | advmod:tlocy
,               | ,          | PUNCT  | punct
hozzám          | hozzá      | PRON   | obl
.               | .          | PUNCT  | punct


In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("magyar_komment_pozitivitas_cp1250.csv", sep=";", encoding="cp1250")

FileNotFoundError: [Errno 2] No such file or directory: 'magyar_komment_pozitivitas_cp1250.csv'

In [48]:
df.sample(5)

,id,pozitivitas,komment
6171,6172,semleges,"Az oldal semleges élmény, semmi extra - nem aj..."
3915,3916,pozitív,A futár minden elvárásomat túlszárnyalta Kösz....
5876,5877,pozitív,"A vendéglátó tényleg remek, köszönöm, minden f..."
1765,1766,pozitív,Az oldal fantasztikus Még visszatérek. tökélet...
7321,7322,pozitív,A bolt baromi jó Kösz. minden flottul ment.


In [49]:
x = df["komment"]
y = df["pozitivitas"]

### Adatok inicializálása

In [50]:
from sklearn.model_selection import train_test_split, cross_val_score

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

### Tokenizálás

In [52]:
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import string

In [53]:
nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle') # Számos szöveget mondatra bontó algoritmus (splitter) van implementálva az NLTKban. A Punkt az egyik, ezt betöltjük.
nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

In [54]:
def tokenize(text):
    vissza = []
    for s, i in enumerate(text):
        doc = nlp(i)
        i = [szo.lemma_ for szo in doc if szo.text not in string.punctuation]
        i = " ".join(i)
        vissza.append(i)
    return vissza


In [55]:
print(len(x_train), len(x_test))

6000 2000


In [57]:
tx_train = tokenize(x_train)
tx_test = tokenize(x_test)

In [58]:
tx_train_out = pd.DataFrame(tx_train, columns=["komment"])
tx_test_out = pd.DataFrame(tx_test, columns=["komment"])
tx_train_out["pozitivitas"] = y_train
tx_test_out["pozitivitas"] = y_test
df_out = pd.concat([tx_train_out, tx_test_out], ignore_index=True)
df_out.sample(5)


,komment,pozitivitas
3477,a szolgáltatás kifejezett rossz mert ismétlődő...,semleges
2586,a film nagyon botrányos késés van a szállítás ...,NaN
693,a szolgáltatás nagyon tökéletes mert elégedett...,NaN
3038,a futár kifejezett pocsék mert nem az kap ami ...,NaN
1734,a vendéglátó marad elfogadható egyszerű rendel...,negatív


### Szózsák

In [59]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [60]:
vectorizer = CountVectorizer(ngram_range=(1,2), token_pattern=r'\b\w+\b', min_df=2)
x_counts = vectorizer.fit_transform(tx_train)
text_x_counts = vectorizer.transform(tx_test)

### Tfidf

In [68]:
tfidf = TfidfTransformer()
x_tfidf = tfidf.fit_transform(x_counts)
test_x_tfidf = tfidf.transform(text_x_counts)

### SelectPercentile

In [69]:
from sklearn.feature_selection import SelectPercentile, chi2

In [70]:
selector = SelectPercentile(score_func=chi2, percentile=44)
x_selected = selector.fit_transform(x_tfidf, y_train)
test_x_selected = selector.transform(test_x_tfidf)

### Modell

In [71]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [72]:
LGR = LogisticRegression()
model = LGR.fit(x_selected, y_train)
predictions = model.predict(x_selected)
test_predictions = model.predict(test_x_selected)

In [73]:
score = cross_val_score(model, x_selected, y_train, cv=5)
print("Pontosság:", accuracy_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Pontosság: 1.0
              precision    recall  f1-score   support

     negatív       1.00      1.00      1.00       776
     pozitív       1.00      1.00      1.00       814
    semleges       1.00      1.00      1.00       410

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

